<a href="https://colab.research.google.com/github/NUELBUNDI/Principles_Data_Science/blob/main/Week4_Exploring_Data_ThursdayCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Include libraries to be used
%matplotlib inline
import pandas as pd
import seaborn as sns


In [ ]:
#Import dataset
#df = pd.read_excel("http://qrc.depaul.edu/Excel_Files/Presidents.xls") #pip install xlrd
df = pd.read_csv('https://raw.githubusercontent.com/chrisnjunge/DStest/master/soccerratings.csv')
#df = pd.read_csv('https://raw.githubusercontent.com/chrisnjunge/DStest/master/W3Store7.csv')

In [ ]:
#df = df2[["Name","Nationality","Club","Club_Position","Rating","Height","Weight","Birth_Date","Age","Work_Rate","Weak_foot"]]

In [ ]:
df.shape

In [ ]:
def create_agebin(df):
    return pd.cut(df.Age, bins=5, labels=[str(x) for x in range(1, 6)])

cs2 = lambda df: pd.cut(df.Age, bins=5, labels=[str(x) for x in range(1, 6)])

def clean_col(col):
  return col.replace(' ', '').replace('.', '').replace('/', '')

def tweak_alta(df):
  return (df
      .rename(columns=clean_col)
      .assign(Birth_Date=pd.to_datetime(df.Birth_Date),
              AGEBIN=create_agebin,
              #AGEBIN=lambda df_: pd.cut(df_.Age, bins=10, labels=[str(x) for x in range(1, 11)]),
              MONTH=lambda df_: df_.Birth_Date.dt.month,
              QTR=lambda df_: df_.Birth_Date.dt.quarter,
              YEAR=lambda df_: df_.Birth_Date.dt.year,
              DAY=lambda df_: df_.Birth_Date.dt.day_name())
      [["Name","Nationality","Club","Club_Position","Rating","Height","Weight","Birth_Date","Age","AGEBIN","Work_Rate","Weak_foot","MONTH","QTR","YEAR","DAY"]]

  )


In [ ]:
df = tweak_alta(df)

In [ ]:
df.columns

Index(['Name', 'Nationality', 'Club', 'Club_Position', 'Rating', 'Height',
       'Weight', 'Birth_Date', 'Age', 'AGEBIN', 'Work_Rate', 'Weak_foot',
       'MONTH', 'QTR', 'YEAR', 'DAY'],
      dtype='object')

In [ ]:
df.MONTH.value_counts()

In [ ]:
# Categorical Data
df.MONTH.value_counts(dropna=False)

In [ ]:
df.MONTH.value_counts().plot.barh()

In [ ]:
# Numeric (Ordinal)
df.Age.mean()

In [ ]:
df.Age.max()

In [ ]:
df.Age.quantile(.99)

In [ ]:
df.Age.quantile([.6, .8])

In [ ]:
df.Age.describe()

In [ ]:
df.Age.plot.hist()

In [ ]:
# Are there outliers
df.Age.plot.box()

In [ ]:
# can also describe a dataframe
df.describe()

In [ ]:
# Relationships - Numeric to Numeric
df.Age.corr(df.Rating)

In [ ]:
df.plot.scatter(x='Age', y='Rating')

In [ ]:
df.plot.scatter(x='Age', y='Rating', alpha=.2)

In [ ]:
# Density
df.plot.hexbin(x='Age', y='Rating', gridsize=20, cmap='viridis')

In [ ]:
sns.relplot(x='Age', y='Rating', data=df)

In [ ]:
df.columns

In [ ]:
sns.relplot(x='Age', y='Rating', data=df, hue='QTR')

In [ ]:
sns.relplot(x='Age', y='Rating', data=df, col='QTR', col_wrap=2, alpha=.1)

In [ ]:
sns.lmplot(x='Age', y='Rating', data=df)

In [ ]:
# lower alpha value
sns.lmplot(x='Age', y='Rating', data=df, x_jitter=1, scatter_kws={'alpha':.1})

In [ ]:
sns.lmplot(x='Age', y='Rating', data=df, x_jitter=1, scatter_kws={'alpha':.1},
          col='QTR', col_wrap=2)

In [ ]:
sns.lmplot(x='Age', y='Rating', data=df, x_jitter=1, scatter_kws={'alpha':.3},
           hue='QTR')

In [ ]:
# combine frequency and scatter plot
sns.jointplot(x='Age', y='Rating', data=df)

In [ ]:
# combine frequency and scatter plot
sns.jointplot(x='Age', y='Rating', data=df, kind='hex')

In [ ]:
sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='RdBu', vmin=-1, vmax=1)

In [ ]:
# 2D Categorical vs Continuous
pd.pivot_table(df, values='Rating', index='AGEBIN', aggfunc='mean')

In [ ]:
# can also do this with a groupby
(df
 .groupby('AGEBIN')
 [['Rating']]
 .mean()
)

In [ ]:
# can also do this with a groupby
(df
 .groupby('AGEBIN')
 ['Rating']
 .describe()
)

In [ ]:
# 2D Categorical vs Continuous
def custom(ser):
    return ser.quantile(.99)

pd.pivot_table(df, values='Rating', index='AGEBIN', 
               aggfunc=['min', 'mean', 'max', custom])

In [ ]:
# 2D Categorical vs Continuous Visualization
sns.boxplot(x='AGEBIN', y='Rating', data=df)

In [ ]:
# 2D Categorical vs Continuous
sns.violinplot(x='Age', y='Rating', data=df)

In [ ]:
# 2D Categorical vs Continuous
sns.boxenplot(x='Age', y='Rating', data=df)

In [ ]:
ax = sns.catplot(x='Age', y='Rating', data=df)

In [ ]:
ax = sns.catplot(x='Age', y='Rating', data=df, alpha=.3)

In [ ]:
# Can add more dimensions with col, row
ax = sns.catplot(x='Age', y='Rating', data=df, kind='boxen', col='QTR', col_wrap=2)

In [ ]:
# 2D Cat vs Cat

In [ ]:
(df
 .pipe(lambda df: pd.crosstab(df.AGEBIN, df.Age))
)

In [ ]:
# percent of occupation by country
(df
 .pipe(lambda df2: pd.crosstab(df2.Weak_foot, df2.Age))
 .pipe(lambda df2: df2.div(df2.sum(axis=1), axis=0))
 .mul(100)
)

In [ ]:
# percent of country by occupation
(df
 .pipe(lambda df2: pd.crosstab(df2.Weak_foot, df2.Age))
 .pipe(lambda df2: df2.div(df2.sum(axis=0), axis=1))
 .mul(100)
)

In [ ]:
# add margin
(df
 .pipe(lambda df2: pd.crosstab(df2.Weak_foot, df2.Age, margins=True))
)

In [ ]:
(df
 .pipe(lambda df2: pd.crosstab(df2.Age, df2.Weak_foot))
 .plot.bar(stacked=True)
)

In [ ]:
# Normalize
(df
 .pipe(lambda df2: pd.crosstab(df2.Age, df2.Weak_foot))
 .pipe(lambda df2: df2.div(df2.sum(axis=1), axis=0))
 .plot.bar(width=1, stacked=True)
)